# Семантический поиск по корпусу текстов

## Определение запроса к корпусу текстов

In [ ]:
QUESTION = 'Что такое LLaMA?'
#QUESTION = 'Что такое авторизация?'
#QUESTION = 'Что такое агрегирование данных?'
#QUESTION = 'Что такое Anonymous?'
#QUESTION = 'Что такое Pretty Good Privacy?'

## Подготовка среды

In [ ]:
# установка библиотек
! pip install -U transformers
! pip install -U accelerate
! pip install sentence_transformers

In [ ]:
# подключение к Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Подготовка модели

In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
	token_embeddings = model_output[0] #First element of model_output contains all token embeddings
	input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
	sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
	sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
	return sum_embeddings / sum_mask

#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained(
	'sentence-transformers/LaBSE')

device = 'cuda'
model = AutoModel.from_pretrained(
	'sentence-transformers/LaBSE').to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Сохранение модели

In [ ]:
# сохранение в локальное хранилище
#model.save_pretrained("drive/MyDrive/Colab Notebooks/semantic_search_QA_telebot/model")

## Получение эмбеддингов из датасета

In [ ]:
# загрузка датасета
knowledge_ds = pd.read_csv("drive/MyDrive/Colab Notebooks/semantic_search_QA_telebot/data/Knowledge_base_df.csv", low_memory=False, encoding = "UTF-8", sep = ",")

In [ ]:
# конкатенация заголовков и текстов
knowledge_ds["sentences"] = (knowledge_ds["title"] + " ") + knowledge_ds["text"]
sentences = [i for i in knowledge_ds["sentences"]]

In [ ]:
# Токенизация текстов
encoded_input = tokenizer(
	sentences,
	padding=True,
	truncation=True,
	max_length=128,
	return_tensors='pt')

In [ ]:
# Вычисление эмбеддингов текстов
with torch.no_grad():
	model_output = model(**encoded_input.to(device))

In [ ]:
# Получение средних значений эмбеддингов
sentence_embeddings = mean_pooling(
	model_output,
	encoded_input['attention_mask'])

## Сохранение эмбеддингов текстов

In [ ]:
sentence_embeddings_df = pd.DataFrame(sentence_embeddings.cpu()).astype('float')
sentence_embeddings_df.to_csv("drive/MyDrive/Colab Notebooks/semantic_search_QA_telebot/data/knowledge_db.csv", encoding="utf-8")

## Загрузка эмбеддингов текстов

In [ ]:
sentence_embeddings_df = pd.read_csv("drive/MyDrive/Colab Notebooks/semantic_search_QA_telebot/data/knowledge_db_128.csv", low_memory=False, encoding = "UTF-8", sep = ",")
sentence_embeddings_df.drop("Unnamed: 0", axis = 1, inplace = True)
sentence_embeddings = torch.cuda.FloatTensor(sentence_embeddings_df.values)

<ipython-input-6-d3c605d52250>:3: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:78.)
  sentence_embeddings = torch.cuda.FloatTensor(sentence_embeddings_df.values)


## Получение эмбеддингов запроса

In [ ]:
question = QUESTION

# Токенизация запроса
encoded_input = tokenizer(
	question,
	padding=True,
	truncation=True,
	max_length=128,
	return_tensors='pt')

# Вычисление эмбеддингов запроса
with torch.no_grad():
	model_output = model(**encoded_input.to(device))

In [ ]:
# Средние значения эмбеддингов запроса
question_embeddings = mean_pooling(
	model_output,
	encoded_input['attention_mask'])

## Поиск наиболее подходящего текста

In [ ]:
# загрузка функции поиска
from sentence_transformers.util import semantic_search

In [ ]:
# применение функции
hits = semantic_search(question_embeddings, sentence_embeddings, top_k=5)

In [ ]:
hits

[[{'corpus_id': 1467, 'score': 0.6242836713790894},
  {'corpus_id': 1466, 'score': 0.5632435083389282},
  {'corpus_id': 488, 'score': 0.5380783081054688},
  {'corpus_id': 1486, 'score': 0.5315546989440918},
  {'corpus_id': 1469, 'score': 0.5273436307907104}]]

In [ ]:
print(sentences[hits[0][0]["corpus_id"]])

Llama-2 Llama-2 - это новая версия большой языковой модели (LLM), разработанная компанией Meta. Она обучена на 2 трлн токенов, что на 40% больше, чем предыдущая версия Llama. Она может выполнять различные задачи, связанные с обработкой текста, такие как суммирование, поиск, творческое письмо, вопросы и ответы, программирование и другие.


## Очистка кэша

In [ ]:
torch.cuda.empty_cache()